Лабораторна робота 2.
Виконала студентка групи ФБ-35 Дедяєва Софія

In [2]:
import urllib.request
import pandas as pd
import os
from datetime import datetime

print("Setup complete")



Setup complete


Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс(номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [3]:
def download_vhi_data(region_id):
    folder_path = 'C:/Dediaieva/Python'
    os.makedirs(folder_path, exist_ok=True)

    try:
        for filename in os.listdir(folder_path):
            if f"_{region_id}_" in filename:
                print(f"[{region_id}] вже завантажено. Пропуск.")
                return
    except Exception as e:
        print(f"Помилка перевірки файлів: {e}")

    base_url = (
        "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?"
        f"country=UKR&provinceID={region_id}&year1=1981&year2=2024&type=Mean"
    )

    try:
        response = urllib.request.urlopen(base_url)
        content = response.read()

        now = datetime.now().strftime("%d%m%Y%H%M%S")
        filename = f"vhi_region_{region_id}_{now}.csv"
        filepath = os.path.join(folder_path, filename)

        with open(filepath, 'wb') as file:
            file.write(content)

        print(f"[{region_id}] Завантажено успішно: {filename}")
    except Exception as e:
        print(f"[{region_id}] Помилка при завантаженні: {e}")

for i in range(1, 26):
    download_vhi_data(i)


[1] вже завантажено. Пропуск.
[2] вже завантажено. Пропуск.
[3] вже завантажено. Пропуск.
[4] вже завантажено. Пропуск.
[5] вже завантажено. Пропуск.
[6] вже завантажено. Пропуск.
[7] вже завантажено. Пропуск.
[8] вже завантажено. Пропуск.
[9] вже завантажено. Пропуск.
[10] вже завантажено. Пропуск.
[11] вже завантажено. Пропуск.
[12] вже завантажено. Пропуск.
[13] вже завантажено. Пропуск.
[14] вже завантажено. Пропуск.
[15] вже завантажено. Пропуск.
[16] вже завантажено. Пропуск.
[17] вже завантажено. Пропуск.
[18] вже завантажено. Пропуск.
[19] вже завантажено. Пропуск.
[20] вже завантажено. Пропуск.
[21] вже завантажено. Пропуск.
[22] вже завантажено. Пропуск.
[23] вже завантажено. Пропуск.
[24] вже завантажено. Пропуск.
[25] вже завантажено. Пропуск.


Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо).Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [35]:
def read_vhi_csv_folder(folder_path):
    columns = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Empty']
    combined_data = []

    for csv_file in os.listdir(folder_path):
        if not csv_file.startswith('vhi_region_'):
            continue

        full_path = os.path.join(folder_path, csv_file)

        try:
            df = pd.read_csv(full_path, header=1, names=columns)

            region_code = int(csv_file.split('_')[2])

            df = df[df['VHI'] != -1]

            df['Region'] = region_code

            combined_data.append(df)

        except Exception as err:
            print(f"Проблема з файлом {csv_file}: {err}")

    if not combined_data:
        print("Жодного файлу не оброблено.")
        return pd.DataFrame()

    result_df = pd.concat(combined_data, ignore_index=True).drop_duplicates().reset_index(drop=True)

    result_df = result_df.drop(columns='Empty')

    result_df['Year'] = result_df['Year'].astype(str)\
        .str.replace('<tt><pre>', '', regex=False)\
        .str.replace('</pre></tt>', '', regex=False)\
        .str.strip()

    result_df = result_df[(result_df['Year'] != '') & (result_df['Week'] != '')]

    result_df['Year'] = result_df['Year'].astype(int)
    result_df['Week'] = result_df['Week'].astype(int)
    result_df['Region'] = result_df['Region'].astype(int)

    return result_df

df_vhi = read_vhi_csv_folder('C:/Dediaieva/Python')
print(df_vhi)


       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      1982     1  0.059  258.24  51.11  48.78  49.95      10
1      1982     2  0.063  261.53  55.89  38.20  47.04      10
2      1982     3  0.063  263.45  57.30  32.69  44.99      10
3      1982     4  0.061  265.10  53.96  28.62  41.29      10
4      1982     5  0.058  266.42  46.87  28.57  37.72      10
...     ...   ...    ...     ...    ...    ...    ...     ...
54669  2024    48  0.135  278.17  55.23  11.86  33.55       9
54670  2024    49  0.133  277.08  57.71  10.86  34.29       9
54671  2024    50  0.130  276.49  59.45   8.68  34.07       9
54672  2024    51  0.128  276.45  62.53   5.55  34.04       9
54673  2024    52  0.129  276.48  66.13   3.71  34.92       9

[54650 rows x 8 columns]


Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [36]:
def rename_regions_to_ukrainian_order(df, region_column="Region"):

    noaa_to_ua_index = {
        1: 24,  2: 25,  3: 26,  4: 27,  5: 2,   6: 3,   7: 8,   8: 7,
        9: 12, 10: 20, 11: 9,  12: 10, 13: 17, 14: 13, 15: 18, 16: 11,
        17: 14, 18: 15, 19: 16, 20: 19, 21: 21, 22: 1,  23: 22, 24: 6,
        25: 5,  26: 23, 27: 4
    }


    df[region_column] = df[region_column].map(noaa_to_ua_index).fillna(df[region_column]).astype(int)
    return df

converted_df = rename_regions_to_ukrainian_order(df_vhi)
print(converted_df)


       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      1982     1  0.059  258.24  51.11  48.78  49.95      20
1      1982     2  0.063  261.53  55.89  38.20  47.04      20
2      1982     3  0.063  263.45  57.30  32.69  44.99      20
3      1982     4  0.061  265.10  53.96  28.62  41.29      20
4      1982     5  0.058  266.42  46.87  28.57  37.72      20
...     ...   ...    ...     ...    ...    ...    ...     ...
54669  2024    48  0.135  278.17  55.23  11.86  33.55      12
54670  2024    49  0.133  277.08  57.71  10.86  34.29      12
54671  2024    50  0.130  276.49  59.45   8.68  34.07      12
54672  2024    51  0.128  276.45  62.53   5.55  34.04      12
54673  2024    52  0.129  276.48  66.13   3.71  34.92      12

[54650 rows x 8 columns]


Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):


o Ряд VHI для області за вказаний рік;

In [40]:
def extract_vhi_range(df, region_ids, year_start, year_end):
    for region_id in region_ids:
        print(f"Region {region_id}:")
        vhi_data = df[(df['Region'] == region_id) & (df['Year'].between(year_start, year_end))]

        if not vhi_data.empty:
            print(vhi_data)
        else:
            print(f"No data available for region {region_id} for years {year_start}-{year_end}")
        print("\n")

extract_vhi_range(converted_df, [3, 7, 14], 2000, 2005)



Region 3:
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
46833  2000     1  0.033  260.79  36.37  29.86  33.12       3
46834  2000     2  0.033  260.71  37.98  30.31  34.14       3
46835  2000     3  0.033  259.80  38.49  35.47  36.98       3
46836  2000     4  0.033  259.54  38.18  40.13  39.16       3
46837  2000     5  0.033  260.15  36.21  42.86  39.54       3
...     ...   ...    ...     ...    ...    ...    ...     ...
47120  2005    48  0.065  267.68  38.99  27.90  33.44       3
47121  2005    49  0.061  264.65  39.38  30.15  34.77       3
47122  2005    50  0.057  263.70  40.59  25.81  33.20       3
47123  2005    51  0.055  263.21  47.96  23.80  35.88       3
47124  2005    52  0.054  262.60  56.48  23.88  40.18       3

[292 rows x 8 columns]


Region 7:
       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
51207  2000     1  0.030  260.15  33.49  28.46  30.98       7
51208  2000     2  0.029  259.92  33.08  30.02  31.55       7
51209  2000     3  0.028

o Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;

In [42]:
def vhi_statistics(df, regions, years):
    for region in regions:
        for year in years:
            subset = df[(df['Region'] == region) & (df['Year'] == year) & (df['VHI'] != -1)]

            if subset.empty:
                print(f"No data for Region {region} in {year}\n")
                continue

            vhi_values = subset['VHI']
            print(f"Region {region}, Year {year}:")
            print(f"   Minimum VHI:  {vhi_values.min():.2f}")
            print(f"   Maximum VHI:  {vhi_values.max():.2f}")
            print(f"   Mean VHI:     {vhi_values.mean():.2f}")
            print(f"   Median VHI:   {vhi_values.median():.2f}")
            print()

vhi_statistics(converted_df, [3, 7, 14], [2000, 2005])


Region 3, Year 2000:
   Minimum VHI:  17.40
   Maximum VHI:  69.82
   Mean VHI:     45.83
   Median VHI:   43.62

Region 3, Year 2005:
   Minimum VHI:  31.86
   Maximum VHI:  76.84
   Mean VHI:     50.55
   Median VHI:   50.19

Region 7, Year 2000:
   Minimum VHI:  9.36
   Maximum VHI:  68.45
   Mean VHI:     41.39
   Median VHI:   40.36

Region 7, Year 2005:
   Minimum VHI:  38.47
   Maximum VHI:  77.32
   Mean VHI:     57.93
   Median VHI:   54.79

Region 14, Year 2000:
   Minimum VHI:  17.83
   Maximum VHI:  53.87
   Mean VHI:     39.53
   Median VHI:   38.89

Region 14, Year 2005:
   Minimum VHI:  31.95
   Maximum VHI:  72.12
   Mean VHI:     49.22
   Median VHI:   45.05



o Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [46]:
def drought_report(df, percent_threshold):
    total_regions = df['Region'].nunique()
    drought_results = {}

    for year in range(df['Year'].min(), df['Year'].max() + 1):
        drought_entries = df[(df['Year'] == year) & (df['VHI'] < 15)]
        affected_regions = drought_entries['Region'].unique()

        if len(affected_regions) > total_regions * (percent_threshold / 100):
            drought_results[year] = {
                'regions': [int(r) for r in affected_regions],
                'vhi': [round(v, 2) for v in drought_entries['VHI']]
            }

    if drought_results:
        print(f"Years with extreme drought affecting more than {percent_threshold}% of the regions in Ukraine:\n")
        for year, values in drought_results.items():
            print(f"Year: {year}")
            print(f"Regions: {values['regions']}")
            print(f"VHI: {values['vhi']}\n")
    else:
        print(f"No years found where more than {percent_threshold}% of the regions experienced extreme drought.")

drought_report(converted_df, 5)



Years with extreme drought affecting more than 5% of the regions in Ukraine:

Year: 1986
Regions: [13, 3]
VHI: [14.07, 12.45, 12.57, 14.99]

Year: 1993
Regions: [13, 3]
VHI: [14.43, 13.1, 14.71, 11.34, 8.02, 6.26, 7.45, 11.29]

Year: 2000
Regions: [9, 10, 24, 19, 6, 7]
VHI: [12.51, 10.6, 11.2, 12.32, 14.65, 14.89, 12.76, 7.81, 6.49, 6.58, 6.71, 7.56, 9.25, 10.94, 12.28, 14.64, 11.82, 10.81, 10.68, 12.3, 14.24, 13.14, 9.5, 8.14, 9.69, 11.2, 11.36, 12.77, 12.26, 11.28, 11.25, 11.38, 12.91, 14.2, 14.61, 11.33, 9.36, 9.45, 9.73, 11.45, 14.29]

Year: 2007
Regions: [11, 14, 27, 12]
VHI: [11.44, 7.78, 6.44, 6.12, 6.11, 5.94, 6.36, 7.61, 9.45, 11.7, 11.88, 9.21, 7.08, 5.9, 5.52, 5.85, 6.82, 8.86, 11.91, 14.98, 14.23, 13.79, 13.41, 13.28, 14.36, 12.41, 12.23, 12.99, 13.33, 12.88, 12.63, 12.96, 13.48, 14.05, 14.41, 14.81]

